# Log lightGBM metrics to neptune

![lightGBM neptune.ml integration](../_static/images/LightGBM_neptuneML.png)

## Prerequisites
Create your dataset and define model parameters.

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
data = load_wine()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.1)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {'boosting_type': 'gbdt',
              'objective': 'multiclass',
              'num_class': 3,
              'num_leaves': 31,
              'learning_rate': 0.05,
              'feature_fraction': 0.9
              }

## Create `neptune_monitor` callback

In [ ]:
import neptune
from neptunecontrib.monitoring.lightgbm import neptune_monitor

neptune.init(project_qualified_name='USER_NAME/PROJECT_NAME')

neptune.create_experiment()
monitor = neptune_monitor()

## Add `neptune_monitor` callback to `lgb.train`

In [ ]:
gbm = lgb.train(params,
        lgb_train,
        num_boost_round=500,
        valid_sets=[lgb_train, lgb_eval],
        valid_names=['train','valid'],
        callbacks=[monitor],
       )

## Stop Neptune experiment

In [ ]:
neptune.stop()

You can also put everything in the `with` block:

In [ ]:
with neptune.create_experiment():
    ...

## Monitor your lightGBM training in neptune
Now you can watch your lightGBM model training in neptune!

![image](https://gist.githubusercontent.com/jakubczakon/f754769a39ea6b8fa9728ede49b9165c/raw/c58e0555ee290640f856d742cf54ba8a2a0b1979/contrib_lgbm.png)

## Full lightgbm monitor script

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
import neptune
from neptunecontrib.monitoring.lightgbm import neptune_monitor

neptune.init(project_qualified_name='USER_NAME/PROJECT_NAME')

data = load_wine()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.1)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {'boosting_type': 'gbdt',
              'objective': 'multiclass',
              'num_class': 3,
              'num_leaves': 31,
              'learning_rate': 0.05,
              'feature_fraction': 0.9
              }

with neptune.create_experiment():
    monitor = neptune_monitor()

    gbm = lgb.train(params,
            lgb_train,
            num_boost_round=500,
            valid_sets=[lgb_train, lgb_eval],
            valid_names=['train','valid'],
            callbacks=[monitor],
           )